In [1]:
from main import compute_urban_morphometrics
from srai.neighbourhoods import H3Neighbourhood
from srai.plotting.folium_wrapper import plot_all_neighbourhood, plot_neighbours
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf

In [4]:
gdf_sf = geocode_to_region_gdf("Karlsruhe, DE")
regions_gdf = H3Regionalizer(8).transform(gdf_sf)
regionalizer = H3Regionalizer(8, buffer=False)
gdf_h3 = regionalizer.transform(regions_gdf)

# gdf_h3.explore()

In [3]:
metrics = compute_urban_morphometrics(
    gdf_h3,
    "https://download.geofabrik.de/europe/germany/baden-wuerttemberg/karlsruhe-regbez-latest.osm.pbf",
    save_data_path="/mnt/c/Users/isaac/Downloads",
    metrics="courtyard_area"
)

Using cached PBF: karlsruhe-regbez-latest.osm.pbf
Extracting OSM data for the combined area...


Splitting highway networks...
Tagging features with region membership...
Cached preprocessed data (e325c194...)
Saved buildings, networks, and landuse to /mnt/c/Users/isaac/Downloads
Computing metrics for 881f8526c1fffff (1/305)...
Computing metrics for 881faac9c1fffff (2/305)...
Computing metrics for 881faacb23fffff (3/305)...
Computing metrics for 881f8526ddfffff (4/305)...
Computing metrics for 881f8526c3fffff (5/305)...
Computing metrics for 881f8534d3fffff (6/305)...
Computing metrics for 881f85348dfffff (7/305)...
Computing metrics for 881faac9a3fffff (8/305)...
Computing metrics for 881f8534cdfffff (9/305)...
Computing metrics for 881faac959fffff (10/305)...
Computing metrics for 881faacb29fffff (11/305)...
Computing metrics for 881faac9a7fffff (12/305)...
Computing metrics for 881faac9c5fffff (13/305)...
Computing metrics for 881faacb1dfffff (14/305)...
Computing metrics for 881f852641fffff (15/305)...
Computing metrics for 881faac945fffff (16/305)...
Computing metrics for 881f

In [ ]:
metrics.explore()

In [1]:
import geopandas as gpd
import h3
import pandas as pd
from shapely.geometry import Polygon
from main import _select_for_region, _preprocess_osm_data, _get_region_ids, ensure_pbf

pbf_url = "https://download.geofabrik.de/europe/germany/baden-wuerttemberg/karlsruhe-regbez-latest.osm.pbf"
pbf_file = ensure_pbf(pbf_url)

# Build a single-hexagon GeoDataFrame by H3 cell id
target_region_id = "881faac961fffff"
boundary = h3.cell_to_boundary(target_region_id)
# h3 returns (lat, lng) but Shapely needs (lng, lat)
boundary = [(lng, lat) for lat, lng in boundary]
gdf_h3 = gpd.GeoDataFrame(
    {"geometry": [Polygon(boundary)]},
    index=pd.Index([target_region_id], name="region_id"),
    crs=4326,
)

region_ids = _get_region_ids(gdf_h3)
buildings_all, highways_all, vehicles_all, pedestrians_all, landuse_all = (
    _preprocess_osm_data(gdf_h3, pbf_file, region_ids)
)

print(f"Running courtyard_area_metrics for region: {target_region_id}")

buildings_gdf = _select_for_region(buildings_all, target_region_id)
cell_polygon = gdf_h3.loc[target_region_id, "geometry"]

Using cached PBF: karlsruhe-regbez-latest.osm.pbf
Extracting OSM data for the combined area...


Output()

Finished operation in 0:00:25

Splitting highway networks...
Tagging features with region membership...
Cached preprocessed data (b74b0fab...)
Running courtyard_area_metrics for region: 881faac961fffff


In [7]:
from metrics.dimension import perimeter_wall_metrics

result = perimeter_wall_metrics(buildings_gdf, cell_polygon)
result

,geometry,perimeter_wall_mean,perimeter_wall_std,perimeter_wall_sum,perimeter_wall_p10,perimeter_wall_p20,perimeter_wall_p30,perimeter_wall_p40,perimeter_wall_p50,perimeter_wall_p60,perimeter_wall_p70,perimeter_wall_p80,perimeter_wall_p90
0,"POLYGON ((455432.061 5428262.909, 455299.504 5...",518.373102,382.255922,478976.746242,75.691248,131.692804,270.764559,388.051269,439.632603,535.42531,712.963509,801.885492,1047.698425
